In [ ]:
!pip install matplotlib seaborn pandas numpy plotly

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install pyarrow

In [ ]:
crashes_url = 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=download'
df_crashes= pd.read_csv(crashes_url, low_memory= False)


In [ ]:
persons_url = 'https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=download'
df_persons = pd.read_csv(persons_url, low_memory = False)


In [ ]:
df_crashes.head()

In [ ]:

print(df_crashes.shape)

In [ ]:
print(df_crashes.info())

In [ ]:
print(df_crashes.describe(include='all'))

In [ ]:
#Putting date and time in one column
df_crashes["CRASH DATE"] = df_crashes["CRASH DATE"] + " " + df_crashes["CRASH TIME"]
df_crashes["CRASH DATE"] = pd.to_datetime(df_crashes["CRASH DATE"])


From the date column, let us create four more columns,


1.   Crash year, will have only the year of crash.

2.   Crash month will have the month relating to the crash in numeric format.

3.   Crash month name will be the name of the month.

4.   Crash hour will be the hour of the day, in 24hr format, when the crash occurred.


In [ ]:
df_crashes["CRASH YEAR"] = df_crashes["CRASH DATE"].dt.year
df_crashes["CRASH MONTH"] = df_crashes["CRASH DATE"].dt.month
df_crashes["CRASH MONTH NAME"] = df_crashes["CRASH DATE"].dt.strftime('%b')
df_crashes["CRASH HOUR"] = df_crashes["CRASH DATE"].dt.hour
df_crashes["CRASH WEEK"] = df_crashes["CRASH DATE"].dt.strftime('%a')

In [ ]:
df_crashes.columns


We decided to drop some of the columns related to the location that were too specific such as:


1.   Zip code
2.   Longitude and Latitude coordinates
3.   Streets

We also dropped the additional contributing factors and vehicle codes.



In [ ]:
to_drop = ['ZIP CODE', 'LATITUDE', 'LONGITUDE','LOCATION','OFF STREET NAME',
          'CRASH TIME', 'ON STREET NAME','CROSS STREET NAME',
           'CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
           'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
df_clean = df_crashes.drop(to_drop, axis = 1)
df_clean.info()

In [ ]:
df_clean.isna().sum()


In [ ]:
df_clean[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2']].value_counts()


In [ ]:
df_clean['VEHICLE TYPE CODE 1'].value_counts()[df_clean['VEHICLE TYPE CODE 1'].value_counts()> 1000]


In [ ]:
df_clean[['BOROUGH']].value_counts()

For the null values, we decided to:


1.   Replace null values for contributing factors with "Unspecified" cause we have a lot of nulls.
2.   Replace null values for contributing factors and Borough with "UNKOWN" cause we have a lot of nulls.
3.   Drop null values for number of persons injured and killed cause they are very few.



In [ ]:
contributing_factors = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2']
for col in contributing_factors:
    df_clean[col] = df_clean[col].fillna('Unspecified')

vehicle_type_codes = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2','BOROUGH']
for col in vehicle_type_codes:
    df_clean[col] = df_clean[col].fillna('UNKNOWN')

# Drop null values for persons injured and killed
df_clean.dropna(subset=['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED'], inplace=True)

print(df_clean.isna().sum())

In [ ]:
# Dropping duplicate rows
df_clean.drop_duplicates(inplace=True)

*   SPORT UTILITY / STATION WAGON and Station Wagon/Sport Utility Vehicle are treated as different categories, we will rename the former to match the later.
*   The same is observed in the instances of Sedan vs dr 4 sedan.



In [ ]:
df_clean["VEHICLE TYPE CODE 1"] = df_clean["VEHICLE TYPE CODE 1"].replace(["SPORT UTILITY / STATION WAGON"], "Station Wagon/Sport Utility Vehicle")
df_clean["VEHICLE TYPE CODE 2"] = df_clean["VEHICLE TYPE CODE 2"].replace(["SPORT UTILITY / STATION WAGON"], "Station Wagon/Sport Utility Vehicle")
df_clean['VEHICLE TYPE CODE 1'] = df_clean['VEHICLE TYPE CODE 1'].replace(["4 dr sedan"], "Sedan")
df_clean['VEHICLE TYPE CODE 2'] = df_clean['VEHICLE TYPE CODE 2'].replace(["4 dr sedan"], "Sedan")

In [ ]:
# Number of persons injured and killed should be int not float
df_clean['NUMBER OF PERSONS INJURED'] = df_clean['NUMBER OF PERSONS INJURED'].astype(int)
df_clean['NUMBER OF PERSONS KILLED'] = df_clean['NUMBER OF PERSONS KILLED'].astype(int)

In [ ]:
# Standardizing factors and vehicle types to be small
df_clean['VEHICLE TYPE CODE 1'] = df_clean['VEHICLE TYPE CODE 1'].str.lower()
df_clean['VEHICLE TYPE CODE 2'] = df_clean['VEHICLE TYPE CODE 2'].str.lower()
df_clean['CONTRIBUTING FACTOR VEHICLE 1'] = df_clean['CONTRIBUTING FACTOR VEHICLE 1'].str.lower()
df_clean['CONTRIBUTING FACTOR VEHICLE 2'] = df_clean['CONTRIBUTING FACTOR VEHICLE 2'].str.lower()


In [ ]:
# Check if there is negative values
print(df_clean[df_clean['NUMBER OF PERSONS INJURED'] < 0].shape)
print(df_clean[df_clean['NUMBER OF PERSONS KILLED'] < 0].shape)
print(df_clean[df_clean['NUMBER OF PEDESTRIANS INJURED'] < 0].shape)
print(df_clean[df_clean['NUMBER OF PEDESTRIANS KILLED'] < 0].shape)
print(df_clean[df_clean['NUMBER OF CYCLIST INJURED'] < 0].shape)
print(df_clean[df_clean['NUMBER OF CYCLIST KILLED'] < 0].shape)

We classified the collision as safe if no one was injured or killed and dangerous if else.

In [ ]:
def conditions(df):
    if (df['NUMBER OF PERSONS INJURED'] == 0) and (df['NUMBER OF PERSONS KILLED'] == 0) :
        return 0
    else:
        return 1

df_clean['IsDanger'] = df_clean.apply(conditions, axis=1)

Question 1:
How does the rate of collisions change by each year?

We find that collisions rose dramatically in 2013 and then dipped in 2020 (probably because of lockdown)

In [ ]:
# Crash volume trend by year
crash_volume_by_year = df_crashes.groupby('CRASH YEAR')['COLLISION_ID'].count()

plt.figure(figsize=(12, 6))
sns.lineplot(x=crash_volume_by_year.index, y=crash_volume_by_year.values)
plt.title('Crash Volume by Year')
plt.xlabel('Year')
plt.ylabel('Crash Count')
plt.tight_layout()
plt.show()


Question 2:
How does the rate of collisions change by the hour?

We find that collisions peak between 6pm and 7pm because this is the rush hour in which workers and employees leave work in NYC.

In [ ]:
hourly_crash = df_clean.groupby("CRASH HOUR").size().reset_index(name="Number Of Collisions")
palette_x= ["blue"]
palette_y= ["orange"]
plt.figure(figsize=(13,7))
sns.barplot(data=hourly_crash, x="CRASH HOUR", y="Number Of Collisions", palette=palette_y)
plt.ylabel("Number Of Collisions", fontweight="bold", fontsize=13)
plt.xlabel("Hour Of The Day", fontweight="bold", fontsize=13)
plt.title("Number Of Collisions During Different Hours Of The Day",
          fontweight="bold", fontsize=14)

plt.show()

Question 3:
What are the top contributing factors of dangerous collisions?

Driver's attention or distraction is the main factor for a very large number of the collisions.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assume df_clean is your initial DataFrame

# Use a more memory-efficient approach to get the top 5 factors
# 1. Filter for dangerous collisions first
dangerous_collisions = df_clean[df_clean['IsDanger'] == 1]

# 2. Melt the two factor columns into a single column called 'Factor'
# This transforms the data from wide to long format.
all_factors = pd.melt(dangerous_collisions, 
                      value_vars=['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2'], 
                      value_name='Factor')

# 3. Remove 'unspecified' values from the new 'Factor' column
all_factors = all_factors[all_factors['Factor'] != 'unspecified']

# 4. Now, perform a single value_counts() to get the top 5 across both original columns
top_5_factors = all_factors['Factor'].value_counts().nlargest(5)

# Plotting remains the same
plt.figure(figsize=(10, 6))
sns.barplot(x=top_5_factors.index, y=top_5_factors.values, palette='viridis')
plt.title('Top 5 Contributing Factors in Dangerous Collisions')
plt.xlabel('Contributing Factor')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


# Persons dataset

In [ ]:

df_persons.head()

In [ ]:
print(df_persons.shape)

In [ ]:
df_persons.info()

In [ ]:
print(df_persons.describe(include='all'))

In [ ]:
df_persons.columns

In [ ]:
to_drop = ['CRASH_DATE','CRASH_TIME','VEHICLE_ID']
df_persons_clean = df_persons.drop(to_drop, axis = 1)
df_persons_clean.info()

In [ ]:
df_persons_clean.isna().sum()

In [ ]:
df_persons_clean["EJECTION"].value_counts()


For the null values, we decided to:


1.   Replace null values for contributing factors with "Unspecified" cause we have a lot of nulls.
2.   Replace null values for other categorical fetures with "Unkown" cause we have a lot of nulls.
3.   Drop null values for PersonID cause they are very few.
4.   Replace null values for sex with "U"
5.   Replace null values for age with Median



In [ ]:

contributing_factors_persons = ['CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2']
for col in contributing_factors_persons:
    df_persons_clean[col] = df_persons_clean[col].fillna('Unspecified')

categorical_cols_to_fill_unknown = [
    'EJECTION', 'EMOTIONAL_STATUS', 'BODILY_INJURY', 'POSITION_IN_VEHICLE',
    'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION', 'COMPLAINT', 'PED_ROLE'
]
for col in categorical_cols_to_fill_unknown:
    df_persons_clean[col] = df_persons_clean[col].fillna('Unknown')
df_persons_clean['PERSON_SEX'] = df_persons_clean['PERSON_SEX'].fillna('U')
df_persons_clean['PERSON_AGE'] = df_persons_clean['PERSON_AGE'].fillna(df_persons_clean['PERSON_AGE'].median())

df_persons_clean.dropna(subset=['PERSON_ID'], inplace=True)

print(df_persons_clean.isna().sum())

In [ ]:
# Dropping duplicate rows
df_persons_clean.drop_duplicates(inplace=True)

In [ ]:
# Removing outliers: we assume age should be in the range of 0, 125
# Checking number of rows that have age greater than 125 or less than 0 then removing them
print(df_persons_clean[df_persons_clean['PERSON_AGE'] > 125].shape)
print(df_persons_clean[df_persons_clean['PERSON_AGE'] < 0].shape)
df_persons_clean = df_persons_clean[df_persons_clean['PERSON_AGE'] <= 125]
df_persons_clean = df_persons_clean[df_persons_clean['PERSON_AGE'] >= 0]

Question 4:

Is there a gender that has a higher rate of collisions? Msh asdena 7aga :)

Men have double the number of collisions as Women.

In [ ]:
# Plot bar chart for sex without 'U'
sex_counts = df_persons_clean[df_persons_clean['PERSON_SEX'] != 'U']['PERSON_SEX'].value_counts()
plt.figure(figsize=(8, 6))
sns.barplot(x=sex_counts.index, y=sex_counts.values, palette='viridis')
plt.title('Number of Persons by Sex')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.show()


Question 5: What are the distribution of Persons Involved in Collisions by Age Group?

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assume df_persons_clean is your initial DataFrame

# 1. Data Preparation: Ensure 'PERSON_AGE' is a numeric type and remove invalid values.
# This prevents errors during binning.
# We convert errors to NaN (Not a Number) and then drop those rows for this analysis.
ages = pd.to_numeric(df_persons_clean['PERSON_AGE'], errors='coerce').dropna()

# 2. Define Age Bins and Labels
# We'll group ages into common demographic categories.
# `right=False` means the bin includes the left edge (e.g., [18, 26) includes 18 but not 26)
bins = [0, 18, 26, 36, 46, 56, 66, np.inf]
labels = ['0-17', '18-25', '26-35', '36-45', '46-55', '56-65', '66+']

# 3. Create the Age Bins using pd.cut
# This function segments and sorts the age data into the bins defined above.
age_bins = pd.cut(ages, bins=bins, labels=labels, right=False)

# 4. Count the number of persons in each age bin
age_distribution = age_bins.value_counts().sort_index()

# 5. Plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x=age_distribution.index, y=age_distribution.values, palette='viridis')
plt.title('Distribution of Persons Involved in Collisions by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Number of Persons (Count)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


We joined the two datasets using an inner join on collision_id

In [ ]:
df_merged = pd.merge(df_clean, df_persons_clean, on='COLLISION_ID', how='inner')
print("Shape of the merged DataFrame:", df_merged.shape)
print("First 5 rows of the merged DataFrame:")
print(df_merged.head())

Post-Integration Cleaning


In [ ]:
print(df_merged.shape)

In [ ]:
print(df_merged.info())



In [ ]:
print(df_merged.isnull().sum())


In [ ]:
print(df_merged.describe(include='all'))


In [ ]:
print(df_merged.duplicated().sum())

6- what is the Relationship Between Safety Equipment and Collision Danger? 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assume df_persons_clean is your initial DataFrame

# 1. Filter out 'Unknown' values from the 'SAFETY_EQUIPMENT' column.
# We also handle other potential "unknown" variations like 'UNSPECIFIED'.
unknown_values = ['Unknown', 'UNSPECIFIED']
df_filtered = df_merged[~df_merged['SAFETY_EQUIPMENT'].isin(unknown_values)]

# 2. Create the plot.
# A countplot is perfect for showing the number of occurrences.
plt.figure(figsize=(12, 8))
sns.countplot(
    x='SAFETY_EQUIPMENT', 
    hue='IsDanger', 
    data=df_filtered, 
    palette='viridis'
)

# 3. Add titles and labels for clarity.
plt.title('Relationship Between Safety Equipment and Collision Danger', fontsize=16)
plt.xlabel('Safety Equipment', fontsize=12)
plt.ylabel('Count of Persons Involved', fontsize=12)

# 4. Improve readability of the x-axis labels.
plt.xticks(rotation=45, ha='right')

# 5. Adjust the legend.
# The default 'IsDanger' legend might just show 0 and 1. We can make it more descriptive.
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles, ['Not Dangerous (0)', 'Dangerous (1)'], title='Collision Danger')

plt.tight_layout()
plt.show()


Data looks good with no duplicates and not that many nulls so we will not be changing anything.

Which vehicle types are most frequently involved in accidents with cyclists?

7- what are the Top 10 Vehicle Types Most Frequently Involved in Accidents with Cyclists?

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Filter the DataFrame to include only accidents where at least one cyclist was injured or killed.
cyclist_accidents_df = df_merged[(df_merged['NUMBER OF CYCLIST INJURED'] > 0) | (df_merged['NUMBER OF CYCLIST KILLED'] > 0)].copy()

# Combine the vehicle type columns to get a single list of all vehicles involved in these accidents.
# We use pd.concat to stack 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2'.
vehicles_involved = pd.concat(
    [cyclist_accidents_df['VEHICLE TYPE CODE 1'], cyclist_accidents_df['VEHICLE TYPE CODE 2']],
    ignore_index=True
)

# Count the occurrences of each vehicle type and get the top 10.
# We drop null values and filter out any 'Unknown' types for a cleaner chart.
top_vehicle_types = vehicles_involved.dropna().loc[lambda x: x != 'Unknown'].value_counts().nlargest(10)

# Create a figure and axes for the plot
plt.figure(figsize=(12, 8))

# Generate the horizontal bar plot
sns.barplot(
    x=top_vehicle_types.values,
    y=top_vehicle_types.index,
    orient='h',
    palette='magma'
)

# Add titles and labels for clarity
plt.title('Top 10 Vehicle Types Most Frequently Involved in Accidents with Cyclists', fontsize=16)
plt.xlabel('Number of Accidents', fontsize=12)
plt.ylabel('Vehicle Type', fontsize=12)

# Adjust layout to prevent labels from being cut off
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Convert the column to string to handle both numbers and 'Unknown'
#df_merged['VEHICLE_ID'] = df_merged['VEHICLE_ID'].astype(str)

# Use Parquet instead of CSV. It is much faster and smaller,
# keeping your repo size low (GitHub has a 100MB limit).
# Now try saving again
df_merged.to_parquet('crashes.parquet')

In [ ]:
import pandas as pd

# Load the file
df = pd.read_parquet('crashes.parquet')

# Print all column names
print("\n--- COLUMN NAMES IN YOUR FILE ---")
print(df.columns.tolist())
print("---------------------------------\n")

8- How does the severity of accidents vary across different boroughs?

In [ ]:
# Question 1: Accident Severity Across Different Boroughs
# Memory-efficient approach: Select only needed columns first, then get unique collisions
print("Getting unique collisions (this may take a moment)...")

# Select only the columns we need for this analysis
cols_needed = ['COLLISION_ID', 'BOROUGH', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']
df_subset = df_merged[cols_needed].copy()

# Use groupby().first() instead of drop_duplicates() - more memory efficient
collisions_unique = df_subset.groupby('COLLISION_ID').first().reset_index()

# Filter out missing borough values (now on much smaller dataset)
valid_borough_mask = collisions_unique['BOROUGH'].notna() & (collisions_unique['BOROUGH'] != '')
borough_severity = collisions_unique.loc[valid_borough_mask].copy()

# Group by borough and calculate average injuries and deaths per accident
borough_stats = borough_severity.groupby('BOROUGH').agg({
    'NUMBER OF PERSONS INJURED': 'mean',
    'NUMBER OF PERSONS KILLED': 'mean',
    'COLLISION_ID': 'count'  # Total number of accidents per borough
}).round(3)

borough_stats.columns = ['Avg_Injuries', 'Avg_Deaths', 'Total_Accidents']

# Sort by average deaths (descending) to see which boroughs have deadlier accidents
borough_stats = borough_stats.sort_values('Avg_Deaths', ascending=False)

print("Average Injuries and Deaths per Accident by Borough:")
print(borough_stats)
print(f"\nTotal unique collisions analyzed: {len(borough_severity):,}")


In [ ]:
# Grouped Bar Chart: Average Injuries vs. Deaths per Borough
boroughs = borough_stats.index
avg_injuries = borough_stats['Avg_Injuries']
avg_deaths = borough_stats['Avg_Deaths']

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(14, 8))

# Set the width of the bars
x = np.arange(len(boroughs))
width = 0.35

# Create the bars
bars1 = ax.bar(x - width/2, avg_injuries, width, label='Average Injuries per Accident', 
               color='#3498db', alpha=0.8)
bars2 = ax.bar(x + width/2, avg_deaths, width, label='Average Deaths per Accident', 
               color='#e74c3c', alpha=0.8)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

# Customize the chart
ax.set_xlabel('Borough', fontsize=12, fontweight='bold')
ax.set_ylabel('Average Count per Accident', fontsize=12, fontweight='bold')
ax.set_title('Accident Severity Across Boroughs: Average Injuries vs. Deaths per Accident', 
             fontsize=16, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(boroughs, rotation=45, ha='right')
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

# Additional insights
print("\n=== Key Insights ===")
print(f"Borough with highest average injuries per accident: {borough_stats['Avg_Injuries'].idxmax()} "
      f"({borough_stats['Avg_Injuries'].max():.3f})")
print(f"Borough with highest average deaths per accident: {borough_stats['Avg_Deaths'].idxmax()} "
      f"({borough_stats['Avg_Deaths'].max():.3f})")
print(f"\nNote: This shows severity (average per accident), not total volume.")
print(f"Borough with most total accidents: {borough_stats['Total_Accidents'].idxmax()} "
      f"({borough_stats['Total_Accidents'].max():,} accidents)")


9- What are the most common pedestrian actions leading to accidents?

In [ ]:
pedestrian_injuries = df_merged[df_merged['NUMBER OF PEDESTRIANS INJURED'] > 0].copy()

# Filter out missing values and 'Unknown' entries for cleaner analysis
pedestrian_actions = pedestrian_injuries[pedestrian_injuries['PED_ACTION'].notna() & 
                                         (pedestrian_injuries['PED_ACTION'] != 'Unknown') &
                                         (pedestrian_injuries['PED_ACTION'] != '')].copy()

# Count the occurrences of each pedestrian action
action_counts = pedestrian_actions['PED_ACTION'].value_counts()

print(f"Total accidents with pedestrian injuries: {len(pedestrian_injuries):,}")
print(f"Accidents with valid PED_ACTION data: {len(pedestrian_actions):,}")
print(f"\nTop 15 Most Common Pedestrian Actions Leading to Injuries:")
print(action_counts.head(15))


In [ ]:
# Horizontal Bar Chart: Most Common Pedestrian Actions Leading to Accidents
# Get top actions (limit to top 15 for readability)
top_actions = action_counts.head(15)

# Create figure and axes
plt.figure(figsize=(14, 10))

# Generate the horizontal bar plot
sns.barplot(
    x=top_actions.values,
    y=top_actions.index,
    orient='h',
    palette='viridis'
)

# Add titles and labels
plt.title('Most Common Pedestrian Actions Leading to Accidents (Filtered for Cases with Injuries)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Number of Accidents with Pedestrian Injuries', fontsize=12, fontweight='bold')
plt.ylabel('Pedestrian Action', fontsize=12, fontweight='bold')

# Add value labels on bars
for i, v in enumerate(top_actions.values):
    plt.text(v + 50, i, f'{v:,}', va='center', fontsize=10)

# Adjust layout
plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n=== Summary Statistics ===")
print(f"Total unique pedestrian actions: {len(action_counts)}")
print(f"Most common action: '{action_counts.index[0]}' with {action_counts.iloc[0]:,} occurrences")
print(f"Percentage of total: {(action_counts.iloc[0] / len(pedestrian_actions) * 100):.2f}%")


10- What is the relationship between being ejected from a vehicle and the severity of bodily injury?

In [ ]:
# Question 4: Relationship between Ejection and Bodily Injury Severity
# Filter out missing values and 'Unknown' entries for cleaner analysis
ejection_injury_data = df_merged[['EJECTION', 'BODILY_INJURY']].copy()
ejection_injury_data = ejection_injury_data[
    (ejection_injury_data['EJECTION'].notna()) & 
    (ejection_injury_data['EJECTION'] != 'Unknown') &
    (ejection_injury_data['BODILY_INJURY'].notna()) & 
    (ejection_injury_data['BODILY_INJURY'] != 'Unknown')
]

# Create a crosstab to show the relationship
crosstab = pd.crosstab(ejection_injury_data['EJECTION'], ejection_injury_data['BODILY_INJURY'], normalize='index') * 100

print("Proportion of Bodily Injury Types by Ejection Status (%):")
print(crosstab.round(2))
print(f"\nTotal records analyzed: {len(ejection_injury_data):,}")


In [ ]:
# Heatmap: Relationship between Ejection and Bodily Injury Severity
plt.figure(figsize=(14, 8))
sns.heatmap(
    crosstab, 
    annot=True, 
    fmt='.1f', 
    cmap='YlOrRd', 
    cbar_kws={'label': 'Percentage (%)'},
    linewidths=0.5,
    linecolor='gray'
)
plt.title('Heatmap: Proportion of Bodily Injury Types by Ejection Status', fontsize=16, fontweight='bold')
plt.xlabel('Bodily Injury Type', fontsize=12, fontweight='bold')
plt.ylabel('Ejection Status', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()


Question 11: Which boroughs have the highest accident frequency?

In [ ]:
plt.figure(figsize=(10, 6))

borough_counts = df_merged['BOROUGH'].value_counts().reset_index()
borough_counts.columns = ['Borough', 'Accident Count']

sns.barplot(data=borough_counts, x='Borough', y='Accident Count', palette='magma')
plt.title('Accident Frequency by Borough')
plt.xlabel('Borough')
plt.ylabel('Number of Accidents')
plt.tight_layout()
plt.show()

Question 12: Who is most vulnerable in accidents (Pedestrians, Cyclists, or Motorists)?

In [ ]:
plt.figure(figsize=(10, 6))

injury_counts = pd.DataFrame({
    'Category': ['Pedestrians', 'Cyclists', 'Motorists'],
    'Total Injured': [
        df_merged['NUMBER OF PEDESTRIANS INJURED'].sum(),
        df_merged['NUMBER OF CYCLIST INJURED'].sum(),
        df_merged['NUMBER OF MOTORIST INJURED'].sum()
    ]
})

sns.barplot(data=injury_counts, x='Category', y='Total Injured', palette='coolwarm')
plt.title('Total Injuries by Victim Category')
plt.xlabel('Victim Category')
plt.ylabel('Total Persons Injured')
plt.tight_layout()
plt.show()
